# Strain-desing Workflow

In [31]:
## IMPORTS
#Cobra dependencies
import cobra
from cobra import Reaction, Metabolite
from cobra.io import read_sbml_model, save_matlab_model
from cobra.flux_analysis import production_envelope, flux_variability_analysis
from cobra.flux_analysis.variability import find_essential_genes
from cobra.sampling import sample
from cobra.sampling import OptGPSampler, ACHRSampler
import gurobipy
#Cameo dependencies
import cameo
from cameo.strain_design.deterministic.linear_programming import OptKnock
from cameo import phenotypic_phase_plane
from cameo.visualization.plotting.with_plotly import PlotlyPlotter
from cameo.strain_design.deterministic.flux_variability_based import FSEOF
from cameo.flux_analysis.simulation import lmoma, pfba
#Dependencies for Minimal Cut Sets (MCS) analysis
import straindesign as sd
#Data processing dependencies
import ast
import re
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from itertools import combinations
#Metabolic design helper functions
from pyfastcore import set_medium
#Our own helper functions
import subprocess #to call matlab gcFront execution script
from utils.designFunctions import *  #our own design functions
from utils.importExcelModel import * #to generate GEMs from excel spreadsheets
#widgets
import ipywidgets as widgets
from ipywidgets import Layout, HBox, HTML
#warnings
import warnings

## 1. Set pipeline parameters

In [ ]:
#Set up all parameter input as interactive widgets. 
#Distribute all parameter input at the beginning of the pipeline
'''
quedan por poner en ipywidgets:
    minimum_growth_fraction
    min_production_flux
    TOP_CANDIDATES
    N_OF_DELETIONS
    TOP_DELETION_STRAINS
    N_OF_DELETION_STRAINS
    TOP_STRATEGIES
'''

In [48]:
#for determining max_knock_out_range and max_knock_out_range:
range_ko = widgets.SelectionRangeSlider(
    options=[i for i in range(30)],
    index=(0, 29),
    disabled=False
)
display(HBox([HTML('Knock-Out range'), range_ko]))

In [52]:
for i in range(range_ko.value[0], range_ko.value[1]+1):
    print(i)

3
4
5
6
7
8
9
10
11


In [3]:
#Set the parameters for exploration of all combinations:
max_knock_out_range =  range(2,6,1)
max_knock_out_range = range(12,8,-1)
replicates = 3

#Project-specific parameters
dir_path = '.'
project_name = 'example_ko_results'
experiment = '_'.join(project_name.split('_')[0:-2])
framework_name = 'cameo'
minimum_growth_fraction = 0.1
min_production_flux = 0.05
#WORKFLOW REDIRECTION PARAMETERS
NEED_TO_SET_UP_MEDIA = False
REGULATION_TARGETS = True
CHECK_GROWTH_COUPLING_FEASIBILITY = True
set_up_params = {'dir_path' : dir_path,
                 'project' : project_name,
                 'framework' : framework_name,
                 'min_growth' : minimum_growth_fraction,
                 'max_cl_range' : max_cl_range,
                 'max_knock_out_range' : max_knock_out_range,
                 'replicates' : replicates}

#Parameters for flux sampling strategy generation
sorting_param = 'Normalised_max_Flux' #TO CHOOSE BETWEEN: 'Normalised_max_Flux',               
secondary_param = 'Presence'          #                   'Normalised_max_biomass' or 'Presence'

#Flux Sampling Parameters:
TOP_CANDIDATES = 10
N_OF_DELETIONS = 5
TOP_DELETION_STRAINS = 2
N_OF_DELETION_STRAINS = 6

#Selection Parameters:
TOP_STRATEGIES = 3

## 2. Configure the model according to bioprocess conditions

### 2.1 Model & media specification

In [3]:
#Specify the model and the required parameters
model_path = 'models'
model_filename = 'ecoli_isobutanol.xml'
model=cobra.io.read_sbml_model('/'.join([project_name, model_path, model_filename]))
target_biomass = 'Ec_biomass_iCA1273_core_59p81M'
target_metabolites = ['btd_RR_e']
target_reaction = 'EX_btd_RR_e'
carbon_source = 'EX_ac_e'
#minimize flux ranges to reduce the solution space
model.summary()

Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-19


Metabolite,Reaction,Flux,C-Number,C-Flux
ac_e,EX_ac_e,12.8,2,99.15%
ca2_e,EX_ca2_e,0.001352,0,0.00%
cl_e,EX_cl_e,0.001352,0,0.00%
cobalt2_e,EX_cobalt2_e,0.0009015,0,0.00%
cu2_e,EX_cu2_e,0.0009015,0,0.00%
fe2_e,EX_fe2_e,0.004184,0,0.00%
h_e,EX_h_e,10.15,0,0.00%
k_e,EX_k_e,0.0507,0,0.00%
mg2_e,EX_mg2_e,0.002254,0,0.00%
mn2_e,EX_mn2_e,0.0009015,0,0.00%


### 2.2 Model reduction

In [4]:
#Preprocess the model with user data
configured_model = model.copy()
configured_model.objective = target_biomass
#Specify media if not previously define in a preparation script
if NEED_TO_SET_UP_MEDIA:
    media_definition = {    'EX_ca2_e' : 10,
                            'EX_cl_e' : 10,
                            'EX_co2_e' : 100,
                            'EX_cobalt2_e' : 10,
                            'EX_cu2_e' : 10,
                            'EX_fe2_e' : 10,
                            'EX_fe3_e' : 0,
                            'EX_h_e' : 100,
                            'EX_h2o_e' : 100,
                            'EX_k_e' : 10,
                            'EX_mg2_e' : 10,
                            'EX_mn2_e' : 10,
                            'EX_mobd_e' : 10,
                            'EX_na1_e' : 10,
                            'EX_tungs_e' : 1000,
                            'EX_zn2_e' : 10,
                            'EX_ni2_e' : 10,
                            'EX_sel_e' : 1000,
                            'EX_slnt_e' : 1000,
                            'EX_so4_e' : 10,
                            'EX_nh4_e' : 10,
                            'EX_pi_e' : 10,
                            'EX_cbl1_e' : 0,
                            'EX_o2_e' : 30,
                            carbon_source : 3.78}
    
    set_medium(configured_model, media_definition, inplace=True)

configured_model = purge_non_objective_biomass(configured_model, target_biomass, n_of_biomass_reactions=3)
blocked_reactions = set([r.id for r in configured_model.reactions])-set(get_rxn_with_fva_flux(configured_model))
#remove blocked reactions
print('Removing a total of %d blocked reactions...' % len(blocked_reactions))
configured_model.remove_reactions(blocked_reactions)
#save model to .mat format if it were neccessary to run gcFront after
save_matlab_model(configured_model, '/'.join([dir_path, project_name, "configured_model.mat"]))
configured_model.summary()

Read LP format model from file /tmp/tmpnc_82bwt.lp
Reading time = 0.01 seconds
: 2032 rows, 5898 columns, 23086 nonzeros
Read LP format model from file /tmp/tmp7vb1nn5l.lp
Reading time = 0.01 seconds
: 2032 rows, 5898 columns, 23086 nonzeros
Read LP format model from file /tmp/tmp6snkp6eo.lp
Reading time = 0.01 seconds
: 2032 rows, 5898 columns, 23086 nonzeros
Removing a total of 1119 blocked reactions...


Metabolite,Reaction,Flux,C-Number,C-Flux
ac_e,EX_ac_e,12.8,2,99.15%
ca2_e,EX_ca2_e,0.001352,0,0.00%
cl_e,EX_cl_e,0.001352,0,0.00%
cobalt2_e,EX_cobalt2_e,0.0009015,0,0.00%
cu2_e,EX_cu2_e,0.0009015,0,0.00%
fe2_e,EX_fe2_e,0.004184,0,0.00%
h_e,EX_h_e,10.15,0,0.00%
k_e,EX_k_e,0.0507,0,0.00%
mg2_e,EX_mg2_e,0.002254,0,0.00%
mn2_e,EX_mn2_e,0.0009015,0,0.00%


### 2.3 FSEOF analysis for Up/Dow regulation target suggestion

In [ ]:
if REGULATION_TARGETS:
    fseof = FSEOF(configured_model)
    fseof_result = fseof.run(target=configured_model.reactions.get_by_id(target_reaction))
    fseof_data = process_fseof_result(fseof_result)
    threshold = fseof_data.flux_difference.tolist()[0]*0.2

    regulation_targets =plot_fseof_analysis(fseof_data,
                                            configured_model.reactions.get_by_id(target_reaction),
                                            threshold,
                                            regulation_type = 'U')

    print('If you want, choose a regulation target')
    regulation_target = widgets.RadioButtons(
                            layout={'width': 'max-content'},
                            options = regulation_targets,
                            description='Candidates:',
                            disabled=False
    )
    display(regulation_target)

    print('Now choose the up/down regulation fold you consider to apply:')
    regulation_fold = widgets.IntSlider(
                            min=0,
                            max=10,
                            step=1,
                            description='Test:',
                            disabled=False,
                            continuous_update=False,
                            orientation='horizontal',
                            readout=True,
                            readout_format='d'
    )
    display(regulation_fold)

In [25]:
#PERFORM THE CHANGES IN THE MODEL TO GENERATE THE CONFIGURED MODEL WITH REGULATION MODIFICATIONS
if REGULATION_TARGETS:
    flux_level = fseof_data.loc[regulation_target.value][1]*regulation_fold.value
    if flux_level < 0:
        configured_model.reactions.get_by_id(regulation_target.value).bounds = (-1000, flux_level)
    else:
        configured_model.reactions.get_by_id(regulation_target.value).bounds = (flux_level, 1000)

    print('Reaction %s has been over/under regulated %s times, now bounds are %s' %
         (regulation_target.value, regulation_fold.value, configured_model.reactions.get_by_id(regulation_target.value).bounds))
    #save model to .mat format if it were neccessary to run gcFront after
    save_matlab_model(configured_model, '/'.join([dir_path, project_name, "configured_model.mat"]))
    configured_model.summary()

Reaction ME1 has been over/under regulated 2 times, now bounds are (2.77236, 1000)


Metabolite,Reaction,Flux,C-Number,C-Flux
ac_e,EX_ac_e,12.8,2,99.14%
ca2_e,EX_ca2_e,0.001367,0,0.00%
cl_e,EX_cl_e,0.001367,0,0.00%
cobalt2_e,EX_cobalt2_e,0.0009111,0,0.00%
cu2_e,EX_cu2_e,0.0009111,0,0.00%
fe2_e,EX_fe2_e,0.004229,0,0.00%
h_e,EX_h_e,10.12,0,0.00%
k_e,EX_k_e,0.05124,0,0.00%
mg2_e,EX_mg2_e,0.002278,0,0.00%
mn2_e,EX_mn2_e,0.0009111,0,0.00%


## 3. Check Growth Coupling (GC) feasibility

In [34]:
#IMPLEMENT THE PIPELINE IN check_growth_coupling_feasibility.ipynb
if CHECK_GROWTH_COUPLING_FEASIBILITY:
    display(HTML('<b>Growth Coupling Feasibility Parameters</b>'))
    minimum_yield_from_biomass = widgets.FloatSlider(
                                    value=2.5,
                                    min=0,
                                    max=10.0,
                                    step=0.1,
                                    disabled=False,
                                    continuous_update=False,
                                    orientation='horizontal',
                                    readout=True,
                                    readout_format='.1f'
                                 )
    
    minimum_yield_from_substrate = widgets.FloatSlider(
                                    value=0.05,
                                    min=0,
                                    max=1.0,
                                    step=0.05,
                                    disabled=False,
                                    continuous_update=False,
                                    orientation='horizontal',
                                    readout=True,
                                    readout_format='.2f'
                                 )
    
    #display(minimum_yield_from_biomass)
    display(HBox([HTML('Minimum Product Yield from Biomass'), minimum_yield_from_biomass]))
    
    display(HBox([HTML('Minimum Product Yield from Sustrate'), minimum_yield_from_substrate]))    
    

HTML(value='<b>Growth Coupling Feasibility Parameters</b>')

In [ ]:
#GENERATE CONSTRAINTS FOR FINDING MCS ALLOWING STRONG AND/OR WEAK GROWTH COUPLINGS 
constraints = gc_filtering_constraints_generator(configured_model, target_biomass, target_reaction, carbon_source,
                                                 growth_fraction=minimum_growth_fraction,
                                                 yield_from_biomass=minimum_yield_from_biomass.value,
                                                 yield_from_sustrate=minimum_yield_from_substrate.value)

#DISPLAY METABOLIC SPACES ALLOWING THE DIFFERENT CONSTRAINTS
display(HTML('<b>Metabolic space for a viable bioprocess</b>'))
sd.plot_flux_space( model,
                   (target_biomass,(target_reaction)),
                   constraints=constraints['bioprocess_constraints'])

display(HTML('<b>Metabolic space needed to be removed to enable a strong growth coupling (SGC)</b>'))
sd.plot_flux_space( model,
                   (target_biomass,(target_reaction)),
                   constraints=constraints['coupling_constraints'][0])

display(HTML('<b>Metabolic space needed to be removed to enable a weak growth coupling (WGC)</b>'))
sd.plot_flux_space( model,
                   (target_biomass,(target_reaction)),
                   constraints=constraints['coupling_constraints'][0])

In [ ]:
gc_feasibility_result = check_gc_feasibility(model, constraints)
display(gc_feasibility_result)

## 4. Execute the suitable strategy design algortihm according the GC feasibility results

In [ ]:
#ADD 'minimum_flux' IN THE MATLAB SCRIPT

In [ ]:
#Generate OptKnock results for the specified parameters if the grwth coupling is feasible:
gc_feasible = any(gc_feasibility_result.loc[model.id][:-1])
if gc_feasible:
    #branch-specific variables
    analysis_type = ['gene_candidates','strategies_eval']
    #now the workflow branch for searching gc strategies should start
    
    #FIRST: we execute several optknock runs over the parameter space for all the specified replicates:
    display(HTML('<b>EXECUTING OPTKNOCK SEARCH OF STRATEGIES...</b>'))
    generate_strategies(set_up_params, configured_model, target_biomass, target_reaction, carbon_source, blocked_reactions)
    
    #SECOND: analysis of OptKnock results
    display(HTML('<b>ANALYSIS OF OPTKNOCK RESULTS</b>'))
    ok_raw_df, ok_results_df = analyse_results(set_up_params,configured_model,target_reaction,sorting_param,analysis_type=analysis_type)
    #ranking figure of top reactions in OptKnock result:
    display(HTML('<b>Ranking of top reactions in Optknock designs</b>'))
    ranking_fig = px.bar(results_df, x="Reaction", y=sorting_param, color=secondary_param, color_continuous_scale='Bluered_r')
    ranking_fig.show()
    #strategies performance over the parameter space:
    display(HTML('<b>Performance of OptKnock strategies</b>'))
    deletion_list = raw_df.query("Flux>="+str(min_production_flux))["N_of_deletions"].unique().tolist()
    deletion_list.sort()
    carbon_list = raw_df.query("Flux>="+str(min_production_flux))["C_limit"].unique().tolist()
    carbon_list.sort()
    performance_fig = px.scatter(raw_df.query("Flux>="+str(min_production_flux)),
                                 x="Flux", y="Biomass", color="Biomass",
                                 facet_col="N_of_deletions", facet_row="C_limit",hover_name="Strategy",
                                 category_orders = {"N_of_deletions":deletion_list,
                                                    "C_limit":carbon_list})

    performance_fig.for_each_annotation(lambda a: a.update(text=a.text.replace("N_of_deletions", "dels")))
    performance_fig.show()

    warnings.warn("Sometimes Optknock strategies are not reproducible when applied over the model, please check if they work")
    
    #save figures:
    ranking_fig.write_image('/'.join([dir_path, project_name, "candidate_reactions_reduced_parameter_space.png"]))
    performance_fig.write_image('/'.join([dir_path, project_name, "optknock_strategies_evaluation_reduced_parameter_space.png"]))

    #THIRD: execution of gcFront
    display(HTML('<b>EXECUTING gcFront in MATLAB..</b>'))
    #Generate a file containing basic information of the set up
    set_up_dict = {'experiment' : [experiment],
                   'target' : [target_reaction],
                   'framework' : [framework_name],
                   'biomass_limit' : [biomas_limit],
                   'minimum_flux' : [min_production_flux]}

    set_up_df = pd.DataFrame.from_dict(set_up_dict)
    set_up_df.to_csv('production_target_tf.csv',index=False)
    #Call bash to launch the gcFront script in the background
    subprocess.call("bash launch_gcFront.sh", shell=True)
    print("gcFront was executed, the process could take hours, meanwhile you can explore OptKnock strategies")
    print("We reccomend you to wait for results and analyse strategies in 'Strategies_Analysis.ipynb' notebook")
    
else:
    #if there are no options for finding a GC solution, execute the branch of the workflow using
    #flux sampling to detect non coupled strategies yielding higher product fluxes than the WT

### Flux sampling analysis

In [ ]:
#INCLUDE ALL ANALYSIS IN A FUNCTION THAT WILL BE EXECUTED OR NOT DEPENDING ON THE FEASIBILITY OF GC

In [ ]:
#sampling of WT
print('Performing flux sampling of WT..')
wt_growth = float(configured_model.optimize().objective_value)
flux_fraction = 0.2
growth_threshold = wt_growth*flux_fraction
configured_model.reactions.get_by_id(target_biomass).bounds = (growth_threshold, 1000)
optgp = OptGPSampler(configured_model, processes=8, thinning=1000)
data = optgp.sample(5600)
data_dict = {target_biomass : data[target_biomass],
             target_reaction : data[target_reaction],
             'strain' : ['wt']*5600}

print('DONE!')      
df = pd.DataFrame.from_dict(data_dict)
#generate deletions of top OptKnock strategies and N candidate reactions:
top_ko_strategies =[s for s in raw_df.loc[(raw_df['Flux']>flux_limit) & (raw_df['Biomass']>biomas_limit), 'Strategy'].tolist()]
comb_deletions = list(combinations(results_df.Reaction.values.tolist()[0:TOP_CANDIDATES], 1))
screening_list = comb_deletions+top_ko_strategies
#perfom a screening of deletions by flux sampling for all top genes:
print('Performing flux sampling of top single-gene deletions..')
df = flux_sampling_screening(screening_list, df, configured_model, target_biomass, target_reaction)
        
#generate a resume df
df_plot = pd.DataFrame(columns = ['strain', 'median_flux', 'median_biomass', 'sem_flux'])
strain_list = list(df.strain.unique())
for strain in strain_list:
    # Group and calculate the mean and sem
    median_flux = df.loc[df['strain']==strain, target_reaction].median()
    sem_flux =  df.loc[df['strain']==strain, target_reaction].sem()
    median_biomass = df.loc[df['strain']==strain, target_biomass].median()
    df_plot.loc[strain_list.index(strain)] = [strain, median_flux, median_biomass, sem_flux]
    

print('DONE!')
#generate N trees of deletions of top N reactions:
TOP_N_DELETIONS = df_plot.sort_values(by='median_flux', ascending=False)[:TOP_DELETION_STRAINS].strain.tolist()
comb_deletions = []
for i in range(N_OF_DELETIONS+1):
    if i>1:
        deletions = [comb 
                     for comb in combinations(df_plot.sort_values(by='median_flux', ascending=False)[:N_OF_DELETION_STRAINS].strain.tolist(), i)
                     if any([top in comb for top in TOP_N_DELETIONS]) 
                     and 'wt' not in comb]
        comb_deletions += deletions

#If the flux sampling has ended before gcFront, wait until the output of the script is written:
from os.path import exists
import time

gcfront_results_fn = '/'.join([dir_path, project_name, 'gcfront_strategies.csv'])
running_trials_left = 3
gcfront_output = os.path.exists(gcfront_results_fn)

while not gcfront_output and running_trials_left > 0:
    #if Matlab not running, it means that previous run has ended without output
    subprocess.run ("ps -eo 'tty,pid,comm' | grep ^? > daemons.txt", shell=True)
    f = open("daemons.txt", "r")
    matlab_finished = 'MATLAB' not in f.read()
    
    if matlab_finished and running_trials_left == 0:
        print('An error ocurred: gcFront could not end with an ouput!')
        gcfront_output = False
        
    if matlab_finished and running_trials_left > 0:
        #If so run again 'launch_GDLS.sh' and wait for file
        print('Matlab ended without output, try again!')
        subprocess.call("bash launch_gcFront.sh", shell=True)
        running_trials_left -= 1
        
    #wait a maximum of 1 hour to check again if matlab is running
    wait_for_file(fn=gcfront_results_fn, max_wait_sec=3600*1*1, check_interval = 60.0)
    
if gcfront_output:
    gcfront_strategies_df = pd.read_csv(gcfront_results_fn)
    gcfront_strategies_df = gcfront_strategies_df.loc[gcfront_strategies_df["GrowthRate"] >= growth_threshold]
    gcfront_strategies_df.sort_values(by='ProductFlux', axis=0, ascending=False, inplace=True)
    gcfront_strategies = [ s.split(' ') for s in gcfront_strategies_df.ReactionDeletions.tolist()]
    #Add top gcFront strategies to the rest of the combinations
    for s in gcfront_strategies[:TOP_STRATEGIES]:
        #replace analogous deletions separated by '/' with only the first one:
        s = [r.split('/')[0] for r in s]
        comb_deletions.append(s)
        
print('Performing flux sampling of top single-gene deletion combinations, a total of %s strategies' % str(len(comb_deletions)))   
df = flux_sampling_screening(comb_deletions, df, configured_model, target_biomass, target_reaction)
#save results:
sampling_fn = '/'.join([set_up_params['dir_path'],set_up_params['project'],experiment+'_validation_'+str(N_OF_DELETIONS)+'_deletion_sampling.csv'])
df.to_csv(sampling_fn, index=False)
df = pd.read_csv(sampling_fn)
print('DONE!')

#generate a resume df
print('Generating file with flux sampling results of the different strategies')
df_plot = pd.DataFrame(columns = ['strain', 'median_flux', 'median_biomass', 'sem_flux'])
strain_list = list(df.strain.unique())
for strain in strain_list:
    # Group and calculate the mean and sem
    median_flux = df.loc[df['strain']==strain, target_reaction].median()
    sem_flux =  df.loc[df['strain']==strain, target_reaction].sem()
    median_biomass = df.loc[df['strain']==strain, target_biomass].median()
    df_plot.loc[strain_list.index(strain)] = [strain, median_flux, median_biomass, sem_flux]
df_plot

Performing flux sampling of WT..
DONE!
Performing flux sampling of top single-gene deletions..
Screening a total of 17 strategies using flux sampling, this is going to take a while...
11200
16800
22400
28000
33600
39200
44800
50400
56000
61600


/home/alvaro/miniconda3/envs/strain-design/lib/python3.9/site-packages/cobra/sampling/optgp.py:174: RuntimeWarning:

Mean of empty slice.

/home/alvaro/miniconda3/envs/strain-design/lib/python3.9/site-packages/numpy/core/_methods.py:181: RuntimeWarning:

invalid value encountered in true_divide



high <= 0
MALATE-DEH-RXN-HACD8-DMPPS-RXN0-884 deletions are lethal
high <= 0
RXN-14273-MALATE-DEH-RXN-RXN0-884-DMPPS deletions are lethal
high <= 0
MALATE-DEH-RXN-CDPREDUCT-RXN-RXN0-884-DMPPS deletions are lethal
high <= 0
MALATE-DEH-RXN-RXN0-884-DMPPS-ACOAD4f deletions are lethal
high <= 0
MALATE-DEH-RXN-R03857-1.18.1.2-RXN-DMPPS deletions are lethal
high <= 0
MALATE-DEH-RXN-1.18.1.2-RXN-DMPPS-RXN-14272 deletions are lethal
high <= 0
MALATE-DEH-RXN-URA-PHOSPH-RXN-ASPDECARBOX-RXN-HACD8 deletions are lethal
DONE!
Waiting for file ./halomonas_PHA_secretion_ko_results/gcfront_strategies.csv
Waiting for file ./halomonas_PHA_secretion_ko_results/gcfront_strategies.csv


In [ ]:
#VISUALIZE RESULT AND SAVE IT
fig = px.scatter(df_plot, x="strain", y="median_flux", color="median_biomass", error_y="sem_flux")

fig.update_layout({"xaxis": {'visible': True, 
                             'showticklabels': True, 
                             'tickangle':45,
                             'tickfont':{'size':10}}
                  })

fig.show()
fig.write_image('/'.join([dir_path, project_name, framework_name, "candidate_reactions_reduced_parameter_space.png"]))

In [ ]:
best_fs = [s.split('-') 
           for s in df_plot.sort_values(by='median_flux', ascending=False).strain.tolist()]
           
display_KO_candidates_results(configured_model, best_fs, target_biomass, target_reaction, 'cameo', media=None)

# 